# Set the environment

In [27]:
%pip install opendatasets


^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


## 1. Libraries

In [2]:
import os
import opendatasets as od
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

## 2. Download Dataset

In [3]:
dataset_url = "https://www.kaggle.com/datasets/arshkon/linkedin-job-postings?resource=download"
download_path = "./data"

In [4]:
#enter username & key downloaded from https://kaggle.com/me/account create new api token
od.download(dataset_url, data_dir=download_path)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle username:Your Kaggle Key:Downloading linkedin-job-postings.zip to ./data\linkedin-job-postings


100%|██████████| 44.9M/44.9M [00:03<00:00, 14.7MB/s]


In [10]:
!tree 

Folder PATH listing for volume Resources
Volume serial number is 14B8-25CB
D:.
����data
    ����linkedin-job-postings
        ����company_details
        ����job_details
        ����maps


In [11]:
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print(f"{subindent}{f}")

list_files('.') 

./
    kaggle_data_extraction.ipynb
    README.md
    data/
        linkedin-job-postings/
            job_postings.csv
            company_details/
                companies.csv
                company_industries.csv
                company_specialities.csv
                employee_counts.csv
            job_details/
                benefits.csv
                job_industries.csv
                job_skills.csv
                salaries.csv
            maps/
                industries.csv
                skills.csv


In [25]:
def get_size(start_path = '.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            if f.endswith('.csv'):
                fp = os.path.join(dirpath, f)
                total_size += os.path.getsize(fp)
    return total_size

def print_file_sizes(start_path = '.'):
    for dirpath, dirnames, filenames in os.walk(start_path):
        print(f"Directory: {dirpath}")
        for f in filenames:
            if f.endswith('.csv'):
                fp = os.path.join(dirpath, f)
                size = os.path.getsize(fp)
                print(f"    {f} - {size} bytes")
        print()  # Print a newline for readability

# Set the start path to your data directory
data_directory = 'data/'  # Adjust this to the path where your data resides
print_file_sizes(data_directory)

Directory: data/

Directory: data/linkedin-job-postings
    job_postings.csv - 134845243 bytes

Directory: data/linkedin-job-postings\company_details
    companies.csv - 10953429 bytes
    company_industries.csv - 390677 bytes
    company_specialities.csv - 2036857 bytes
    employee_counts.csv - 416749 bytes

Directory: data/linkedin-job-postings\job_details
    benefits.csv - 840280 bytes
    job_industries.csv - 666212 bytes
    job_skills.csv - 925814 bytes
    salaries.csv - 731493 bytes

Directory: data/linkedin-job-postings\maps
    industries.csv - 6292 bytes
    skills.csv - 679 bytes



In [26]:
def print_csv_row_counts(start_path='.'):
    for dirpath, _, filenames in os.walk(start_path):
        for f in filenames:
            if f.endswith('.csv'):
                file_path = os.path.join(dirpath, f)
                try:
                    df = pd.read_csv(file_path)
                    print(f"{file_path}: {len(df.index)} rows")
                except pd.errors.EmptyDataError:
                    print(f"{file_path}: Empty")
                except Exception as e:
                    print(f"{file_path}: Failed to read ({e})")

# Set the start path to your data directory
data_directory = 'data/'  # Adjust this to the path where your data resides
print_csv_row_counts(data_directory)

data/linkedin-job-postings\job_postings.csv: 33246 rows
data/linkedin-job-postings\company_details\companies.csv: 11361 rows
data/linkedin-job-postings\company_details\company_industries.csv: 12601 rows
data/linkedin-job-postings\company_details\company_specialities.csv: 78405 rows
data/linkedin-job-postings\company_details\employee_counts.csv: 14275 rows
data/linkedin-job-postings\job_details\benefits.csv: 29325 rows
data/linkedin-job-postings\job_details\job_industries.csv: 44091 rows
data/linkedin-job-postings\job_details\job_skills.csv: 56591 rows
data/linkedin-job-postings\job_details\salaries.csv: 13352 rows
data/linkedin-job-postings\maps\industries.csv: 229 rows
data/linkedin-job-postings\maps\skills.csv: 35 rows


In [14]:
df_posting = pd.read_csv("./data/linkedin-job-postings/job_postings.csv", nrows=100)
df_posting.head(5)

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped
0,3757940104,553718.0,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,NaN,5250.00,NaN,MONTHLY,Full-time,"Little River, SC",...,NaN,Entry level,NaN,1.699090e+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101
1,3757940025,2192142.0,Shipping & Receiving Associate 2nd shift (Beav...,Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",...,NaN,NaN,NaN,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085420
2,3757938019,474443.0,"Manager, Engineering",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",...,NaN,NaN,Bachelor's Degree in Mechanical Engineering pr...,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085644
3,3757938018,18213359.0,Cook,descriptionTitle\n\n Looking for a great oppor...,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",...,NaN,Entry level,NaN,1.699080e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461
4,3757937095,437225.0,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",275834.0,NaN,205956.0,YEARLY,Full-time,United States,...,NaN,Mid-Senior level,NaN,1.699090e+12,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346


In [15]:
df_companies = pd.read_csv("./data/linkedin-job-postings/company_details/companies.csv")
df_companies.head(5)

,company_id,name,description,company_size,state,country,city,zip_code,address,url
0,1009,IBM,"At IBM, we do more than work. We create. We cr...",7.0,NY,US,"Armonk, New York",10504,International Business Machines Corp.,https://www.linkedin.com/company/ibm
1,1016,GE HealthCare,Every day millions of people feel the impact o...,7.0,0,US,Chicago,0,-,https://www.linkedin.com/company/gehealthcare
2,1021,GE Power,"GE Power, part of GE Vernova, is a world energ...",7.0,NY,US,Schenectady,12345,1 River Road,https://www.linkedin.com/company/gepower
3,1025,Hewlett Packard Enterprise,Official LinkedIn of Hewlett Packard Enterpris...,7.0,Texas,US,Houston,77389,1701 E Mossy Oaks Rd Spring,https://www.linkedin.com/company/hewlett-packa...
4,1028,Oracle,We’re a cloud technology company that provides...,7.0,Texas,US,Austin,78741,2300 Oracle Way,https://www.linkedin.com/company/oracle


In [16]:
df_company_industry = pd.read_csv("./data/linkedin-job-postings/company_details/company_industries.csv")
df_company_industry.head(5)

,company_id,industry
0,81149246,Higher Education
1,10033339,Information Technology & Services
2,6049228,Accounting
3,2641066,Electrical & Electronic Manufacturing
4,96649998,Marketing & Advertising


In [17]:
df_employee_counts = pd.read_csv("./data/linkedin-job-postings/company_details/employee_counts.csv")
df_employee_counts.head(5)

,company_id,employee_count,follower_count,time_recorded
0,81149246,6,91,1.692645e+09
1,10033339,3,187,1.692645e+09
2,6049228,20,82,1.692645e+09
3,2641066,45,2336,1.692645e+09
4,96649998,0,2,1.692645e+09


In [18]:
df_benefits = pd.read_csv("./data/linkedin-job-postings/job_details/benefits.csv")
df_benefits.head(5)

,job_id,inferred,type
0,3690843087,0,Medical insurance
1,3690843087,0,Dental insurance
2,3690843087,0,401(k)
3,3690843087,0,Paid maternity leave
4,3690843087,0,Disability insurance


In [20]:
df_job_industries = pd.read_csv("./data/linkedin-job-postings/job_details/job_industries.csv")
df_job_industries.head(5)

,job_id,industry_id
0,3378133231,68
1,3497509795,96
2,3690843087,47
3,3691775263,112
4,3691779379,80


In [21]:
df_job_skills = pd.read_csv("./data/linkedin-job-postings/job_details/job_skills.csv")
df_job_skills.head(5)

,job_id,skill_abr
0,3690843087,ACCT
1,3690843087,FIN
2,3691763971,MGMT
3,3691763971,MNFC
4,3691775263,MGMT


In [22]:
df_salaries = pd.read_csv("./data/linkedin-job-postings/job_details/salaries.csv")
df_salaries.head(5)

,salary_id,job_id,max_salary,med_salary,min_salary,pay_period,currency,compensation_type
0,1,3378133231,30.0,NaN,22.0,HOURLY,USD,BASE_SALARY
1,2,3690843087,65000.0,NaN,55000.0,YEARLY,USD,BASE_SALARY
2,3,3691794313,22.0,NaN,19.0,HOURLY,USD,BASE_SALARY
3,4,3691795389,70000.0,NaN,68000.0,YEARLY,USD,BASE_SALARY
4,5,3691797089,22.0,NaN,18.0,HOURLY,USD,BASE_SALARY


In [23]:
df_industries = pd.read_csv("./data/linkedin-job-postings/maps/industries.csv")
df_industries.head(5)

,industry_id,industry_name
0,1,Defense and Space Manufacturing
1,3,Computer Hardware Manufacturing
2,4,Software Development
3,5,Computer Networking Products
4,6,"Technology, Information and Internet"


In [24]:
df_skills = pd.read_csv("./data/linkedin-job-postings/maps/skills.csv")
df_skills.head(5)

,skill_abr,skill_name
0,PRCH,Purchasing
1,SUPL,Supply Chain
2,PR,Public Relations
3,SCI,Science
4,STRA,Strategy/Planning
